<a href="https://colab.research.google.com/github/wayneotemah/Learing-TensorFlow/blob/main/Recurrent_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import datetime
print(f"Notebook last run (end-to-end): {datetime.datetime.now()}")

Notebook last run (end-to-end): 2023-07-03 11:38:04.172067


In [3]:
# Check for GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-892dfd9e-768e-8aaf-5326-7598cca1c3bd)


In [4]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-07-03 11:38:06--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-07-03 11:38:06 (90.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



# Get the data

In [5]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [6]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-07-03 11:38:10--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.128, 172.253.115.128, 172.253.122.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-07-03 11:38:10 (118 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



# Become one with the data

In [7]:
import pandas as pd

test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

In [8]:
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [9]:
# shuffle the training data
train_df_shuffled = train_df.sample(frac = 1,random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [10]:

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [11]:
# how many examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [12]:
# how many total samples
len(train_df),len(test_df)

(7613, 3263)

In [13]:
# visualize some random training examples
import random
random_index = random.randint(0,len(train_df)-5)

for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")


Target: 1 (real disaster)
Text:
In this fragile global economy considering the devastation the alternatives would cause ... it's the best reason .. https://t.co/zwVyisyP2B

---

Target: 0 (not real disaster)
Text:
The Danger and Excitement of Underwater Cave Diving http://t.co/8c3fPloxcr http://t.co/cBGZ9xuN2k

---

Target: 0 (not real disaster)
Text:
Get å£150 free bets when you register at Boylesports Casino #Luck #Destroy http://t.co/zildpvKNXP http://t.co/5yDb4s13pF

---

Target: 0 (not real disaster)
Text:
@CaraJDeIevingnc the bomb impact ratio hit beyond kyle js

---

Target: 1 (real disaster)
Text:
Nashville Theater Attack: Will Gun Grabbers Now Demand ÛÏHatchet Control?Û  http://t.co/OyoGII97yH

---



# making validation dataset


In [14]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility

In [15]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [16]:
# View the first 10 training sentences and their labels
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

#Converting text into numbers

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization # after TensorFlow 2.6

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                     split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [18]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [19]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [20]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [21]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Your brain is particularly vulnerable to trauma at two distinct ages http://t.co/KnBv2YtNWc @qz @TaraSwart @vivian_giang      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  33, 1890,    9, 4957, 2616,    5,  359,   17,  116, 5800, 6247,
           1, 9408, 7796, 6994]])>

In [23]:

# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


# create embedding layer with tensorflow

In [24]:
tf.random.set_seed(42)

from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer = "uniform",
                             input_length = max_length,
                             name="embedding_1"
                             )

embedding

In [25]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Corey Robinson having some solid reps at RT although as I type this he got beat on lightning dip by T. Walker      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03392627, -0.01062075,  0.00246213, ..., -0.02465214,
         -0.00298184, -0.0244553 ],
        [ 0.00422049, -0.00522494, -0.01781549, ...,  0.03534278,
         -0.04936395, -0.00720326],
        [ 0.02457721, -0.04261488,  0.01385075, ..., -0.02887039,
          0.01795227, -0.03393389],
        ...,
        [-0.01946995,  0.00174582,  0.02327419, ..., -0.04286006,
         -0.02103782,  0.00721109],
        [ 0.03489264, -0.04799175, -0.00624678, ..., -0.04332314,
          0.00675154, -0.03821006],
        [-0.04716926,  0.04533335,  0.03372641, ...,  0.04543385,
          0.03489988, -0.03773383]]], dtype=float32)>

In [26]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.03392627, -0.01062075,  0.00246213,  0.01064172, -0.01847845,
        0.04885915,  0.03983724, -0.02949215,  0.01824277,  0.00267915,
        0.01292485, -0.00461346,  0.01890555, -0.00500283,  0.03907876,
       -0.03704528, -0.01190977, -0.02284287, -0.0243089 , -0.04970584,
        0.0166302 , -0.03509299,  0.02186188, -0.02673351, -0.02376578,
       -0.0131196 , -0.04737722, -0.00737799, -0.04989822, -0.02422445,
       -0.04364859, -0.04786373,  0.03148318, -0.0258311 , -0.02187729,
        0.03365244, -0.01395575, -0.00698493, -0.02252558,  0.01654451,
       -0.00745787,  0.04018855,  0.02785918, -0.01358273, -0.00585647,
        0.00871987,  0.04502142, -0.02333192,  0.01181101,  0.02433592,
       -0.03169622, -0.01329626,  0.02709422, -0.04897988,  0.00745592,
        0.01199507,  0.03380031, -0.04526805, -0.04436241,  0.03463682,
        0.01695034, -0.01116469,  0.01104559, -0.01244054, -0.03682663,
        0.011785

In [27]:
sample_embed[0][0].shape

TensorShape([128])

#Modelling a text dataset

## Model 0: Making the baseline model

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
#Evaluate out baseline model

baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [30]:
# make predictions

baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## evaluation function for our experiments

Using the following metrics
> Accuracy

> Precision

> Recall

> F1 score

In [31]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [32]:
baseline_results = calculate_results(val_labels,baseline_preds)

In [33]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [34]:
## Model1: A simple dense model

from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

In [35]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name="model_1_dense")


In [36]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [37]:
# compile model
model_1.compile(loss="binary_crossentropy",
                optimizer  = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [38]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230703-113818
Epoch 1/5
215/215 [==============================] - 24s 73ms/step - loss: 0.6108 - accuracy: 0.6926 - val_loss: 0.5367 - val_accuracy: 0.7546
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4421 - accuracy: 0.8189 - val_loss: 0.4691 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3472 - accuracy: 0.8610 - val_loss: 0.4589 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2858 - accuracy: 0.8917 - val_loss: 0.4638 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2389 - accuracy: 0.9121 - val_loss: 0.4761 - val_accuracy: 0.7874


In [39]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4761 - accuracy: 0.7874


[0.4760805368423462, 0.787401556968689]

In [40]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

24/24 [==============================] - 0s 2ms/step


array([[0.3991895 ],
       [0.7450078 ],
       [0.9977633 ],
       [0.1086658 ],
       [0.11065951],
       [0.9370551 ],
       [0.9168582 ],
       [0.9927758 ],
       [0.9668267 ],
       [0.2663794 ]], dtype=float32)

In [41]:
# Turn prediction probabilities into single-dimension tensor of floats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [42]:
# Calculate model_1 metrics
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7914920592553047,
 'recall': 0.7874015748031497,
 'f1': 0.7846966492209201}

In [43]:
# Is our simple Keras model better than our baseline model?
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

### Visualizing the models embeding layer

In [44]:
# get the vocabulary from the text vextorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [45]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [46]:

# Get weight metrix of emdedding layer
# numerical rep of each token in our training data, learned for the 5 epochs
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
embed_weights

array([[-0.02682019, -0.02628351,  0.0517611 , ..., -0.02398896,
        -0.00325742, -0.05707677],
       [-0.03313052, -0.05223046,  0.02837171, ..., -0.00490881,
        -0.01851831,  0.03655025],
       [ 0.06138717, -0.01355016,  0.00794036, ..., -0.0340883 ,
         0.06308873, -0.02321191],
       ...,
       [ 0.02072039, -0.03177287, -0.02035631, ..., -0.02832719,
         0.00185981, -0.03772938],
       [ 0.03864992,  0.02644457,  0.08531279, ...,  0.04556694,
         0.07904733, -0.08572797],
       [ 0.09908817,  0.06198273,  0.04514749, ...,  0.0426289 ,
         0.11347993, -0.08951192]], dtype=float32)

In [47]:
# create embedding files
# import io
# out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
# out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

# for index, word in enumerate(words_in_vocab):
#   if index == 0:
#     continue  # skip 0, it's padding.
#   vec = embed_weights[index]
#   out_v.write('\t'.join([str(x) for x in vec]) + "\n")
#   out_m.write(word + "\n")
# out_v.close()
# out_m.close()

#Recurrent Neural Networks(RNN)

## Model 2: LSTM



In [48]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)

# x = layers.LSTM(units = 64,return_sequences=True)(x)
# print(x.shape)
x = layers.LSTM(units = 64)(x)
print(x.shape)
# x = layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name="model_2_LSTM")


(None, 15, 128)
(None, 64)


In [49]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [50]:
#  compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [51]:
# Fit model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20230703-113853
Epoch 1/5
215/215 [==============================] - 15s 53ms/step - loss: 0.2275 - accuracy: 0.9167 - val_loss: 0.5357 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1633 - accuracy: 0.9402 - val_loss: 0.6010 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1285 - accuracy: 0.9523 - val_loss: 0.7352 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1070 - accuracy: 0.9600 - val_loss: 0.8360 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0815 - accuracy: 0.9663 - val_loss: 0.9974 - val_accuracy: 0.7808


In [52]:

# Make predictions on the validation dataset
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10] # view the first 10

24/24 [==============================] - 0s 3ms/step


((762, 1),
 array([[1.7822124e-03],
        [6.5818083e-01],
        [9.9955851e-01],
        [2.3590263e-02],
        [1.9650185e-04],
        [9.9788624e-01],
        [8.3363515e-01],
        [9.9974543e-01],
        [9.9958533e-01],
        [4.7578925e-01]], dtype=float32))

In [53]:
# Round out predictions and reduce to 1-dimensional array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [54]:
# Calculate LSTM model results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7841274438015813,
 'recall': 0.7808398950131233,
 'f1': 0.7782630349987989}

In [55]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model_3.GRU

In [56]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.GRU(units = 64,)(x)
print(x.shape)

outputs = layers.Dense(1,activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs,outputs,name="model_3_GRU")


(None, 15, 128)
(None, 64)


In [57]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [58]:
model_3.compile(loss = "binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [59]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20230703-113919
Epoch 1/5
215/215 [==============================] - 14s 52ms/step - loss: 0.1578 - accuracy: 0.9390 - val_loss: 0.6457 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 2s 12ms/step - loss: 0.0864 - accuracy: 0.9688 - val_loss: 0.8137 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0714 - accuracy: 0.9727 - val_loss: 1.0310 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0665 - accuracy: 0.9750 - val_loss: 1.0574 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0528 - accuracy: 0.9771 - val_loss: 1.1914 - val_accuracy: 0.7756


In [60]:

# Make predictions on the validation dataset
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10] # view the first 10

24/24 [==============================] - 1s 4ms/step


((762, 1),
 array([[4.96605830e-03],
        [6.84514403e-01],
        [9.99795973e-01],
        [1.11062355e-01],
        [1.05821375e-04],
        [9.99722302e-01],
        [8.34420025e-01],
        [9.99929547e-01],
        [9.99843121e-01],
        [9.82575178e-01]], dtype=float32))

In [61]:
# Round out predictions and reduce to 1-dimensional array
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [62]:
# Calculate GRU model results
model_3_results = calculate_results(val_labels,model_3_preds)
model_3_results

{'accuracy': 77.55905511811024,
 'precision': 0.7758405894864461,
 'recall': 0.7755905511811023,
 'f1': 0.7744092523940911}

In [63]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [64]:
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7841274438015813,
 'recall': 0.7808398950131233,
 'f1': 0.7782630349987989}

## Model 4: Bidirectiona-LSTM

In [65]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)

# x = layers.Bidirectional(layers.LSTM(64,return_sequences = True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)

outputs = layers.Dense(1,activation="sigmoid")(x)

model_4 = tf.keras.Model(inputs,outputs,name="mode_4_Bidirectional")


In [66]:
model_4.summary()

Model: "mode_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,37

In [67]:
# compile model

model_4.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [68]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_4_Bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_Bidirectional/20230703-114004
Epoch 1/5
215/215 [==============================] - 16s 55ms/step - loss: 0.1085 - accuracy: 0.9692 - val_loss: 0.9030 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0578 - accuracy: 0.9764 - val_loss: 1.1459 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0480 - accuracy: 0.9777 - val_loss: 1.3087 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0430 - accuracy: 0.9804 - val_loss: 1.3276 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0376 - accuracy: 0.9812 - val_loss: 1.5934 - val_accuracy: 0.7730


In [69]:
# Make predictions on the validation dataset
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs.shape, model_4_pred_probs[:10] # view the first 10

24/24 [==============================] - 1s 3ms/step


((762, 1),
 array([[3.3338842e-04],
        [6.8208867e-01],
        [9.9999011e-01],
        [1.3088836e-01],
        [1.0769818e-05],
        [9.9996054e-01],
        [9.8435009e-01],
        [9.9999619e-01],
        [9.9999177e-01],
        [9.9778783e-01]], dtype=float32))

In [70]:
# conver to labes
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [71]:
# cal the results

model_4_results = calculate_results(val_labels,model_4_preds)
model_4_results

{'accuracy': 77.29658792650919,
 'precision': 0.7753400570774207,
 'recall': 0.7729658792650919,
 'f1': 0.7705764257839071}

##model 5: conv1D

In [72]:
# Test out the embedding, 1D convolutional and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sentence into embedding
conv_1d = layers.Conv1D(filters=32, kernel_size=5, activation="relu") # convolve over target sequence 5 words at a time
conv_1d_output = conv_1d(embedding_test) # pass embedding through 1D convolutional layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # get the most important features
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [73]:

# See the outputs of each layer
embedding_test


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01699921, -0.02731089,  0.02147043, ..., -0.00282034,
          0.03320658, -0.00688276],
        [ 0.04388433,  0.00321121, -0.01629474, ...,  0.03554908,
         -0.01242181, -0.00659055],
        [-0.04600838, -0.05335682, -0.0031643 , ...,  0.01974924,
         -0.01183496,  0.01593925],
        ...,
        [ 0.00188448, -0.04117733,  0.04467565, ..., -0.00265229,
          0.01328473, -0.03310703],
        [ 0.00188448, -0.04117733,  0.04467565, ..., -0.00265229,
          0.01328473, -0.03310703],
        [ 0.00188448, -0.04117733,  0.04467565, ..., -0.00265229,
          0.01328473, -0.03310703]]], dtype=float32)>

In [74]:
embedding_test[0]

<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[ 0.01699921, -0.02731089,  0.02147043, ..., -0.00282034,
         0.03320658, -0.00688276],
       [ 0.04388433,  0.00321121, -0.01629474, ...,  0.03554908,
        -0.01242181, -0.00659055],
       [-0.04600838, -0.05335682, -0.0031643 , ...,  0.01974924,
        -0.01183496,  0.01593925],
       ...,
       [ 0.00188448, -0.04117733,  0.04467565, ..., -0.00265229,
         0.01328473, -0.03310703],
       [ 0.00188448, -0.04117733,  0.04467565, ..., -0.00265229,
         0.01328473, -0.03310703],
       [ 0.00188448, -0.04117733,  0.04467565, ..., -0.00265229,
         0.01328473, -0.03310703]], dtype=float32)>

In [75]:
embedding_test[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.01699921, -0.02731089,  0.02147043,  0.01261622,  0.00902732,
       -0.02352925, -0.00362623, -0.03467106, -0.02352916,  0.0728562 ,
        0.03720927, -0.0312915 ,  0.0631123 , -0.04914119, -0.05157744,
       -0.09905483,  0.01459522,  0.03806022, -0.01965585, -0.01990279,
        0.04312429,  0.04977294,  0.01358348,  0.0595008 , -0.02797062,
       -0.03200522, -0.01323894,  0.04407383,  0.01763532,  0.03232373,
       -0.05110563, -0.01734836, -0.05892402, -0.00560912, -0.04206459,
       -0.00285601, -0.05703748, -0.01591394,  0.05384917, -0.05632011,
        0.0289918 , -0.0557876 , -0.04028368, -0.00095016, -0.05275638,
       -0.00977303, -0.03649249,  0.0205288 ,  0.06667897, -0.05320939,
        0.09432141, -0.07343613, -0.01039201, -0.04667415,  0.00859085,
       -0.02184291,  0.11040296,  0.00128786,  0.06971162, -0.01979079,
       -0.0077729 , -0.03930482, -0.11959752,  0.01664422,  0.02532183,
       -0.012076

In [76]:
conv_1d_output[:1]


<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[2.74667852e-02, 1.30990995e-02, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 2.56200917e-02, 1.06538072e-01,
         2.34080516e-02, 0.00000000e+00, 4.92547676e-02, 1.26237348e-02,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.27912101e-02,
         0.00000000e+00, 5.18294014e-02, 2.32214946e-02, 1.13968618e-01,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.18348198e-02,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.07698059e-02],
        [7.30595505e-03, 0.00000000e+00, 4.00160700e-02, 0.00000000e+00,
         0.00000000e+00, 6.69886023e-02, 7.56476587e-03, 0.00000000e+00,
         1.66316964e-02, 0.00000000e+00, 3.22809666e-02, 6.74372641e-05,
         0.00000000e+00, 3.62360626e-02, 0.00000000e+00, 7.54323509e-03,
         3.59662287e-02, 0.00000000e+00, 1.67026967e-02, 0.00000000e+0

In [77]:
max_pool_output[:1]

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.04189701, 0.0130991 , 0.04001607, 0.        , 0.        ,
        0.0669886 , 0.0680147 , 0.10653807, 0.04828468, 0.04154765,
        0.04925477, 0.02078919, 0.        , 0.03623606, 0.        ,
        0.01401008, 0.03596623, 0.00875872, 0.0167027 , 0.04279121,
        0.01925143, 0.0518294 , 0.02996064, 0.11396862, 0.03230654,
        0.04578731, 0.03010922, 0.01183482, 0.0356311 , 0.03349034,
        0.01798577, 0.05935518]], dtype=float32)>

In [78]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_5_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_5")

# Create 1-dimensional convolutional layer to model sequences
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_5_embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

# Compile Conv1D model
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary of our 1D convolution model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 1)              

In [79]:

# Fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20230703-114036
Epoch 1/5
215/215 [==============================] - 14s 51ms/step - loss: 0.5691 - accuracy: 0.7158 - val_loss: 0.4662 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3449 - accuracy: 0.8564 - val_loss: 0.4619 - val_accuracy: 0.7913
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2169 - accuracy: 0.9216 - val_loss: 0.5126 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1396 - accuracy: 0.9553 - val_loss: 0.5931 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0988 - accuracy: 0.9673 - val_loss: 0.6309 - val_accuracy: 0.7874


In [80]:
# Make predictions with model_5
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.6477961 ],
       [0.71414673],
       [0.99950147],
       [0.1382515 ],
       [0.03099012],
       [0.98088145],
       [0.97784096],
       [0.997879  ],
       [0.9994234 ],
       [0.07859538]], dtype=float32)

In [81]:

# Convert model_5 prediction probabilities to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [82]:
# Calculate model_5 evaluation metrics
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)
model_5_results

{'accuracy': 78.74015748031496,
 'precision': 0.7903907506573856,
 'recall': 0.7874015748031497,
 'f1': 0.7850997256765829}

In [83]:
# Comparbaseline_results
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 6: TF hub pretrained sentence encoder

In [84]:
# Example of pretrained embedding with universal sentence encoder - https://tfhub.dev/google/universal-sentence-encoder/4
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") # load Universal Sentence Encoder
embed_samples = embed([sample_sentence,
                      "When you call the universal sentence encoder on a sentence, it turns it into numbers."])

print(embed_samples[0][:50])

tf.Tensor(
[-0.01157028  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680984  0.05589836 -0.0106873  -0.00597291  0.00639323 -0.01819518
  0.00030813  0.09105888  0.05874644 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865347 -0.04209306  0.02678981  0.03011006  0.00321069
 -0.00337973 -0.04787357  0.0226672  -0.00985925 -0.04063613 -0.01292092
 -0.04666384  0.05630299 -0.03949255  0.00517686  0.02495829 -0.0701444
  0.02871508  0.04947684 -0.00633979 -0.08960192  0.02807118 -0.00808364
 -0.01360602  0.0599865  -0.10361787 -0.05195374  0.00232954 -0.02332531
 -0.03758105  0.03327728], shape=(50,), dtype=float32)


In [88]:
embed_samples[0].shape

TensorShape([512])

In [89]:
# We can use this encoding layer in place of our text_vectorizer and embedding layer
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE")


In [102]:
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
  sentence_encoder_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid",name="outpit_layer")
], name="model_6_USE")

# Compile model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_9 (Dense)             (None, 64)                32832     
                                                                 
 outpit_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [103]:
# Train a classifier on top of pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20230703-124256
Epoch 1/5
215/215 [==============================] - 11s 21ms/step - loss: 0.5029 - accuracy: 0.7894 - val_loss: 0.4487 - val_accuracy: 0.7979
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4146 - accuracy: 0.8137 - val_loss: 0.4372 - val_accuracy: 0.8097
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4000 - accuracy: 0.8232 - val_loss: 0.4329 - val_accuracy: 0.8110
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.3928 - accuracy: 0.8257 - val_loss: 0.4284 - val_accuracy: 0.8123
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.3867 - accuracy: 0.8291 - val_loss: 0.4304 - val_accuracy: 0.8123


In [104]:
# Make predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 11ms/step


array([[0.15269093],
       [0.7363346 ],
       [0.98557115],
       [0.20379016],
       [0.74047834],
       [0.6749691 ],
       [0.981382  ],
       [0.9758007 ],
       [0.9240216 ],
       [0.09614698]], dtype=float32)

In [105]:
# Convert prediction probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [106]:
# Calculate model 6 performance metrics
model_6_results = calculate_results(val_labels, model_6_preds)
model_6_results

{'accuracy': 81.23359580052494,
 'precision': 0.8152105639975652,
 'recall': 0.8123359580052494,
 'f1': 0.8105808468856647}

In [107]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 7: TensorFlow Hub Pretrained Sentence Encoder 10% of the training data

In [110]:
train_sentences_90_percent, train_sentences_10_percent, train_labels_90_percent, train_labels_10_percent = train_test_split(np.array(train_sentences),
                                                                                                                            train_labels,
                                                                                                                            test_size=0.1,
                                                                                                                            random_state=42)

In [111]:
# Check length of 10 percent datasets
print(f"Total training examples: {len(train_sentences)}")
print(f"Length of 10% training examples: {len(train_sentences_10_percent)}")

Total training examples: 6851
Length of 10% training examples: 686


In [113]:
# Check the number of targets in our subset of data
# (this should be close to the distribution of labels in the original train_labels)
pd.Series(train_labels_10_percent).value_counts()

0    415
1    271
dtype: int64

In [114]:
# Clone model_6 but reset weights
model_7 = tf.keras.models.clone_model(model_6)

# Compile model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary (will be same as model_6)
model_7.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_9 (Dense)             (None, 64)                32832     
                                                                 
 outpit_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [115]:
# Fit the model to 10% of the training data
model_7_history = model_7.fit(x=train_sentences_10_percent,
                              y=train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "10_percent_tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/10_percent_tf_hub_sentence_encoder/20230703-125249
Epoch 1/5
22/22 [==============================] - 5s 49ms/step - loss: 0.6704 - accuracy: 0.6968 - val_loss: 0.6535 - val_accuracy: 0.6549
Epoch 2/5
22/22 [==============================] - 1s 53ms/step - loss: 0.6006 - accuracy: 0.7959 - val_loss: 0.5993 - val_accuracy: 0.7336
Epoch 3/5
22/22 [==============================] - 1s 58ms/step - loss: 0.5249 - accuracy: 0.8061 - val_loss: 0.5474 - val_accuracy: 0.7559
Epoch 4/5
22/22 [==============================] - 1s 58ms/step - loss: 0.4609 - accuracy: 0.8294 - val_loss: 0.5136 - val_accuracy: 0.7638
Epoch 5/5
22/22 [==============================] - 2s 94ms/step - loss: 0.4170 - accuracy: 0.8440 - val_loss: 0.4948 - val_accuracy: 0.7717


In [116]:
# Make predictions with the model trained on 10% of the data
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 1s 20ms/step


array([[0.23628135],
       [0.766163  ],
       [0.88382524],
       [0.2742512 ],
       [0.57630914],
       [0.8023414 ],
       [0.7741049 ],
       [0.82500046],
       [0.816476  ],
       [0.11053617]], dtype=float32)

In [117]:
# Convert prediction probabilities to labels
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [118]:
# Calculate model results
model_7_results = calculate_results(val_labels, model_7_preds)
model_7_results

{'accuracy': 77.16535433070865,
 'precision': 0.7762974910710267,
 'recall': 0.7716535433070866,
 'f1': 0.7682834985355502}